In [ ]:
import feedback_forensics as ff
import feedback_forensics.app.plotting.paper as paper_plot
import pathlib
from IPython.display import display, Latex

data_path = pathlib.Path("../forensics-data/feedback-forensics-public-results")
fig_save_path = pathlib.Path("./output/figures")
tex_save_path = pathlib.Path("./output/tex")

# ensure save path exists
fig_save_path.mkdir(parents=True, exist_ok=True)
tex_save_path.mkdir(parents=True, exist_ok=True)

datasets = [
    ["multipref_10k_v3.json", "MultiPref"],
    ["llama4_arena_vs_public_version.json", "Llama 4 Arena vs Public"],
    ["arena", "Chatbot Arena"],
    ["prism", "PRISM"],
]

cache = {}
for dataset_path, dataset_name in datasets:
    dataset = ff.DatasetHandler(cache=cache)
    dataset.add_data_from_path(data_path / dataset_path)

    overall_metrics = dataset.get_overall_metrics()
    annotator_metrics = dataset.get_annotator_metrics()

    metric_name = "strength"
    strength_metrics = annotator_metrics[dataset_path.split(".")[0]]["metrics"][metric_name]

    latex_table = paper_plot.get_latex_top_and_bottom_annotators(
        annotator_metrics=strength_metrics,
        metric_name=metric_name.capitalize(),
        title=f"Encouraged personality traits in {dataset_name}"
    )

    with open(tex_save_path / f"001_top_and_bottom_annotators_{dataset_path.split('.')[0]}.tex", "w") as f:
        f.write(latex_table)



In [ ]:
# Analysis of Arena data
import pandas as pd

dataset_name = "arena"
dataset = ff.DatasetHandler(cache=cache)
dataset.add_data_from_path(data_path / dataset_name)

general_df = dataset.first_handler.df
values = [
    'Creative Writing Prompts',
    'Songwriting Prompts',
    'Resume and Cover Letter Writing',
    'Professional Email Communication',
]
dataset.split_by_col(col="narrower_category", selected_vals=values)

metrics_df = dataset.get_annotator_metrics_df(metric_name="strength", index_col_name="Generate a response that...")

latex_str = paper_plot.get_latex_table_from_metrics_df(
    metrics_df=metrics_df.head(10),
    title="Encouraged personality traits across domains in Chatbot Arena",
)

with open(tex_save_path / "002_writing_tasks_arena.tex", "w", encoding="utf-8") as f:
    f.write(latex_str)

